I am targeting audience for restaurant reviews.

In [1]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Restaurant_Reviews.tsv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "d4rklucif3r/restaurant-reviews",
  file_path,
)

print("First 5 records:", df.head())

/tmp/ipython-input-954164868.py:10: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 59.9k/59.9k [00:00<00:00, 1.02MB/s]

First 5 records:                                               Review  Liked
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [11]:
df[df['Liked']==0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  500 non-null    object
 1   Liked   500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.7+ KB


In [10]:
df[df['Liked']==1].info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 922
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  500 non-null    object
 1   Liked   500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.7+ KB


The classes are balanced, with 500 records for each dislike and like

In [17]:
df[df['Liked']==0].head()

,Review,Liked
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
5,Now I am getting angry and I want my damn pho.,0
6,Honeslty it didn't taste THAT fresh.),0
7,The potatoes were like rubber and you could te...,0


In [16]:
df[df['Liked']==1].head()

,Review,Liked
0,Wow... Loved this place.,1
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
8,The fries were great too.,1
9,A great touch.,1


In [28]:
df['Review'].head()

,Review
0,wow... loved place.
1,crust good.
2,tasty texture nasty.
3,stopped late may bank holiday rick steve recom...
4,selection menu great prices.


Remove punctuation

In [3]:
import string
exclude=string.punctuation

def remove_punc(text):
  return text.translate(str.maketrans('','',exclude))
df['Review'].apply(remove_punc)

,Review
0,Wow Loved this place
1,Crust is not good
2,Not tasty and the texture was just nasty
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
995,I think food should have flavor and texture an...
996,Appetite instantly gone
997,Overall I was not impressed and would not go back
998,The whole experience was underwhelming and I t...


In [4]:
df['Review']=df['Review'].apply(remove_punc)

Remove Chat words

In [5]:
chat_words = {
    "AFATK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime Anywhere Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It Is Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You",
    "ILU": "I Love You",
    "IMHO": "In My Humble Opinion",
    "IMO": "In My Opinion",
    "IRL": "In Real Life",
    "LOL": "Laughing Out Loud",
    "LMAO": "Laughing My Ass Off",
    "OIC": "Oh I See",
    "PITA": "Pain In The Ass",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFMAO": "Rolling On The Floor Laughing My Ass Off",
    "SK8": "Skate",
    "STATS": "Your Sex And Age",
    "ASL": "Age Sex Location",
    "THX": "Thank You",
    "TTFN": "Ta Ta For Now",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours Forever",
    "WB": "Welcome Back",
    "WTF": "What The Fuck",
    "WTG": "Way To Go",
    "WUF": "Where Are You From",
    "W8": "Wait",
    "7K": "Sick Laugher"
}


In [6]:
def chat_conversion(text):
  new_text=[]
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return " ".join(new_text)

In [7]:
df['Review']=df['Review'].apply(chat_conversion)


Regex and Stop words Cleaning

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
negations = {"no", "nor", "not", "never", "n't"}
stop_words = stop_words - negations

def preprocess_for_ngrams(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

df["clean_review"] = df["Review"].astype(str).apply(preprocess_for_ngrams)


In [16]:
df.columns

Index(['Review', 'Liked', 'clean_review'], dtype='object')

Splitting the data for training and testing

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TEXT_COL = "clean_review"
LABEL_COL = "Liked"

df = df[[TEXT_COL, LABEL_COL]].dropna().copy()

X = df[TEXT_COL].astype(str)
y = df[LABEL_COL]


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp
)

print(len(X_train), len(X_val), len(X_test))


700 100 200


Generating BOW , NGRAM and tfidf

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


X_train_text, X_val_text, X_test_text = X_train, X_val, X_test


bow_vec = CountVectorizer(min_df=2, max_df=0.95, ngram_range=(1,1))
ngram_vec = CountVectorizer(min_df=2, max_df=0.95, ngram_range=(1,2))
tfidf_vec = TfidfVectorizer(min_df=2, max_df=0.95, ngram_range=(1,2))

Xtr_bow   = bow_vec.fit_transform(X_train_text)
Xva_bow   = bow_vec.transform(X_val_text)
Xte_bow   = bow_vec.transform(X_test_text)

Xtr_ng    = ngram_vec.fit_transform(X_train_text)
Xva_ng    = ngram_vec.transform(X_val_text)
Xte_ng    = ngram_vec.transform(X_test_text)

Xtr_tfidf = tfidf_vec.fit_transform(X_train_text)
Xva_tfidf = tfidf_vec.transform(X_val_text)
Xte_tfidf = tfidf_vec.transform(X_test_text)


In [19]:

!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 60.4 MB/s eta 0:00:00


Testing Word2Vec with skipgram

In [24]:
from gensim.models import Word2Vec

def simple_tokenize(s):
    return s.lower().split()

train_tokens = [simple_tokenize(t) for t in X_train_text]
val_tokens   = [simple_tokenize(t) for t in X_val_text]
test_tokens  = [simple_tokenize(t) for t in X_test_text]

w2v = Word2Vec(
    sentences=train_tokens,
    vector_size=200,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    negative=10,
    epochs=10
)

def doc_vec(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0).astype(np.float32)

Xtr_w2v = np.vstack([doc_vec(t, w2v) for t in train_tokens])
Xva_w2v = np.vstack([doc_vec(t, w2v) for t in val_tokens])
Xte_w2v = np.vstack([doc_vec(t, w2v) for t in test_tokens])

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def get_avg(y):
    return "binary" if len(pd.unique(y)) == 2 else "macro"

AVG = get_avg(y_train)

def eval_model(name, model, Xtr, ytr, Xte, yte, avg):
    model.fit(Xtr, ytr)
    pred = model.predict(Xte)
    acc = accuracy_score(yte, pred)
    p, r, f1, _ = precision_recall_fscore_support(yte, pred, average=avg, zero_division=0)
    return {"Model": name, "Accuracy": acc, "Precision": p, "Recall": r, "F1": f1}


In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

results = []


results.append(eval_model("Multinomial Navie Bayes (BoW)",   MultinomialNB(), Xtr_bow,   y_train, Xte_bow,   y_test, AVG))
results.append(eval_model("Multinomial Navie Bayes (1-2gram)", MultinomialNB(), Xtr_ng,  y_train, Xte_ng,    y_test, AVG))
results.append(eval_model("Multinomial Navie Bayes (TFIDF 1-2)", MultinomialNB(), Xtr_tfidf, y_train, Xte_tfidf, y_test, AVG))


lr_sparse = LogisticRegression(max_iter=2000)
results.append(eval_model("Logistic Regression (TFIDF 1-2)", lr_sparse, Xtr_tfidf, y_train, Xte_tfidf, y_test, AVG))

svm_sparse = LinearSVC()
results.append(eval_model("LinearSVM (TFIDF 1-2)", svm_sparse, Xtr_tfidf, y_train, Xte_tfidf, y_test, AVG))

lr_dense = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000))
])
results.append(eval_model("Logistic Regression (W2V avg)", lr_dense, Xtr_w2v, y_train, Xte_w2v, y_test, AVG))

svm_dense = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LinearSVC())
])
results.append(eval_model("LinearSVM (W2V avg)", svm_dense, Xtr_w2v, y_train, Xte_w2v, y_test, AVG))

results_df = pd.DataFrame(results).sort_values("F1", ascending=False)
results_df


,Model,Accuracy,Precision,Recall,F1
1,Multinomial Navie Bayes (1-2gram),0.810,0.798077,0.83,0.813725
3,Logistic Regression (TFIDF 1-2),0.810,0.816327,0.80,0.808081
2,Multinomial Navie Bayes (TFIDF 1-2),0.790,0.773585,0.82,0.796117
4,LinearSVM (TFIDF 1-2),0.790,0.773585,0.82,0.796117
0,Multinomial Navie Bayes (BoW),0.790,0.778846,0.81,0.794118
6,LinearSVM (W2V avg),0.745,0.747475,0.74,0.743719
5,Logistic Regression (W2V avg),0.715,0.708738,0.73,0.719212


The Multinomial Navie Bayes Algorithm (1-2 grams) performed the best out of the models, It is was expected as the bigrams capture phrases like very good, not good, highly recomment which a unigram model alone cannot capture.



The Word2Vec seems to be failing here, i cannot capture the natural phrases like "not good". Also we trained it on limited data so that might an issue.

Multinomial Navie Bayes Algorithm is fastest to train on sparse counts then logistic Regrssion and Linear SVM.

Inference is fast on Word2Vec in all of these models.
